In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [3]:
df_w_cluster = pd.read_csv('../Data/raw_df_with_cluster.csv')
features = ['Volatility_6m','CVM Category_Fixed Income'
            ,'CVM Category_Multimarket','Leveraged Anbima_Yes'
interest_cols = ['CVM Category_Fixed Income','CVM Category_Multimarket',
                 'CVM Category_Equities','Liquidity Ratios','age_months'
                 ,'Assets_Flow_6m','Assets_Flow_12m','Assets_Flow_24m'
                 ,'#_Funds_managed', 'Manager_avg_Assets'
                 ,'Management Fee','Return_6m','Return_12m'
                 ,'Return_24m','Volatility_6m','Volatility_12m'
                 ,'Volatility_24m']

# Interactive 4 Plot

In [13]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt

def scatter_plot(Col1,Col2,Col3):
    fig = plt.figure(figsize=(8,6))
    ax = Axes3D(fig)
    x= Col1
    y= Col2
    z= Col3
    for i in df_w_cluster['kmeans_1'].unique():
        xs = df_w_cluster[df_w_cluster['kmeans_1']==i][x].mean()
        ys = df_w_cluster[df_w_cluster['kmeans_1']==i][y].mean()
        zs = df_w_cluster[df_w_cluster['kmeans_1']==i][z].mean()
        size = max(25,min(500,np.mean(df_w_cluster[df_w_cluster['kmeans_1']==i]['Last_Assets'])/3000000))
        ax.scatter(xs, ys, zs, label=str(i),s=size)


    ax.set_title('Type of Mutual Fund in each cluster  ',loc='center', fontsize=22)
    ax.set_xlabel(x, fontsize=13)
    ax.set_ylabel(y, fontsize=13)
    ax.set_zlabel(z, fontsize=13)
    ax.legend(loc = 2)
    plt.show()
    
# Create interactive 
interact(scatter_plot,Col1 =interest_cols,
         Col2 = interest_cols[1:],
         Col3 = interest_cols[2:])

interactive(children=(Dropdown(description='Col1', options=('CVM Category_Fixed Income', 'CVM Category_Multima…

<function __main__.scatter_plot(Col1, Col2, Col3)>

# Interactive Fund Search

In [10]:
risk_dic = {"Conservative":1,"Moderately Conservative":2,"Moderately Agressive":3,
            "Agressive":4,"Very Agressive":5}
fund_type_dic = {'Fixed Income':'CVM Category_Fixed Income',
                 'Balanced':'CVM Category_Multimarket',
                 'Equity':'CVM Category_Equities',
                 "Currency":'CVM Category_FX'}
comp_size_dic = {"Large company":(2/3,1) ,"Medium company":(1/3,2/3),"Small company":(0,1/3),
                 "Any size":(0,1)}

In [20]:
def recommended_funds(Fund_type,Risk_profile,Red_Delay,Min_inves):
    filter_features = ['#_Funds_managed']
    risk_quantile = risk_dic[Risk_profile]/len(risk_dic)     # Risk quantile [0.2, 0.4, 0.6 , 0.8, 1]
    red_delay_min = Red_Delay[0]
    red_delay_max = Red_Delay[1]
    min_inv= Min_inves
#     comp_quant_min = comp_size_dic[cpy_size][0]
#     comp_quant_max = comp_size_dic[cpy_size][1]
    new_df = df_w_cluster.loc[(df_w_cluster['Liquidity Ratios'] >= red_delay_min) &
                 (df_w_cluster['Liquidity Ratios'] <= red_delay_max) &
                 (df_w_cluster['Minimum Investment'] <= min_inv) & 
                 (df_w_cluster[fund_type_dic[Fund_type]] == 1)
                ]  
#     print(red_delay_min,red_delay_max,min_inv,fund_type_dic[Fund_type])
#     display(new_df.groupby(['kmeans_1'])['Volatility_24m'].agg([np.mean,len]))
    vol_quantile_value = new_df['Volatility_24m'].quantile(risk_quantile)
    vol_clust_mean = new_df.groupby(['kmeans_1'])['Volatility_24m'].agg([np.mean]).sort_values(by=['mean'],ascending = False)
    vol_clust_mean['dif'] = vol_clust_mean['mean'].map(lambda x: np.absolute(x-vol_quantile_value))
    final_cluster = vol_clust_mean.sort_values(by=['dif']).index[0]
    print("Final Cluster: " + str(final_cluster))
    print('There are {} results '.format(new_df.shape[0]))
    print("Top 5 returns: ")
    display(new_df[new_df['kmeans_1']==final_cluster][['Name','Management Fee'
                                                       ,'Return_24m','Volatility_24m'
                                                       ]].sort_values(by=['Return_24m'],ascending = False).head())
    return None 

In [19]:
function_interact = interact_manual(recommended_funds,
                Fund_type = fund_type_dic.keys(),
                Risk_profile = risk_dic.keys(),
                Red_Delay=widgets.IntRangeSlider(value=[0, 7], min=0,  max=30, step=1),
                Min_inves=widgets.BoundedIntText(value=1000,min=0, max=500000,step=1, 
                                                 description='Min_inves', disabled=False),
#                 cpy_size = widgets.RadioButtons(options=['Large company', 'Medium company'
#                                                          ,'Small company', 'Any size'], 
#                 description='Company', disabled=False)
               )

interactive(children=(Dropdown(description='Fund_type', options=('Fixed Income', 'Balanced', 'Equity', 'Curren…